# Football World Cup 2022

In [97]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [98]:
dict_table = pd.read_html("https://www.wallchart.io/football/world-cup-2022.html")
df_matches = pd.read_csv("/Users/hilmanyusoh/Documents/PWFWCup 2022/clean_fifa_worldcup_matches.csv")
df_fixture = pd.read_csv("/Users/hilmanyusoh/Documents/PWFWCup 2022/clean_fifa_worldcup_fixture.csv")

# 1.Calculate team strenght

In [99]:
# split dataframe into datahome and dataway
data_home = df_matches[["HomeTeam","HomeGoals","AwayGoals"]]
data_away = df_matches[["AwayTeam","HomeGoals","AwayGoals"]]

In [100]:
# rename columns
data_home = data_home.rename(columns={"HomeTeam": "Team", "HomeGoals": "GoalsScored", "AwayGoals": "GoalsConceded"})
data_away = data_away.rename(columns={"AwayTeam": "Team", "HomeGoals": "GoalsConceded", "AwayGoals": "GaolsScored"})
                         

In [101]:
# concat data_home and data_away, group by team and calculate the mean
df_team_strenght = pd.concat([data_home, data_away], ignore_index=True).groupby('Team').mean()
df_team_strenght

,GoalsScored,GoalsConceded,GaolsScored
Team,,,
Algeria,0.833333,1.461538,1.142857
Angola,0.000000,0.666667,0.500000
Argentina,1.913793,1.148148,1.130435
Australia,1.000000,1.937500,0.666667
Austria,2.285714,1.620690,0.733333
...,...,...,...
Uruguay,2.060606,1.321429,0.826087
Wales,2.000000,0.800000,0.500000
West Germany,2.500000,1.241935,1.500000


## 2 Function predict_point

In [102]:
def predict_points(home, away):
    if home in df_team_strenght.index and away in df_team_strenght.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strenght.at[home,'GoalsScored'] * df_team_strenght.at[away,'GoalsConceded']
        lamb_away = df_team_strenght.at[away,'GoalsScored'] * df_team_strenght.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goal home team
            for y in range(0,11): #number of goal away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(x, lamb_away)
                if x==y:
                    prob_draw +=p
                elif x > y:
                    prob_home +=p
                else:
                    prob_away +=p
                    
        point_home = 3 * prob_home + prob_draw
        point_away = 3 * prob_away + prob_draw
        return (point_home, point_away)
    else:
        return (0, 0)
    
                

## 2.1 Testing function

In [103]:
# Test with matches: Argentina - Mexico, England - United States, Qatar (H) - Equador
predict_points('Argentina', 'Mexico')
predict_points('England', 'United States')
predict_points('Qatar (H)', 'Equador')

(0, 0)

# 3 Predicting World Cup

## 3.1 Group stage

In [104]:
# splitting fixture into group, knockout, quarter, ...
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [105]:
df_fixture_group_48

,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
5,Netherlands,Match 36,Qatar,2022
6,England,Match 3,Iran,2022
7,United States,Match 4,Wales,2022
8,Wales,Match 17,Iran,2022
9,England,Match 20,United States,2022


In [106]:
dict_table[2]

,Team,W,D,L,GF - GA,Pts
0,Senegal,0,0,0,0 - 0,0
1,Qatar,0,0,0,0 - 0,0
2,Ecuador,0,0,0,0 - 0,0
3,Netherlands,0,0,0,0 - 0,0


In [107]:
dict_table[3]

,Team,W,D,L,GF - GA,Pts
0,England,0,0,0,0 - 0,0
1,Iran,0,0,0,0 - 0,0
2,USA,0,0,0,0 - 0,0
3,Wales,0,0,0,0 - 0,0


In [108]:
dict_table[4]

,Team,W,D,L,GF - GA,Pts
0,Argentina,0,0,0,0 - 0,0
1,S.Arabia,0,0,0,0 - 0,0
2,Mexico,0,0,0,0 - 0,0
3,Poland,0,0,0,0 - 0,0


In [109]:
dict_table[5]

,Team,W,D,L,GF - GA,Pts
0,France,0,0,0,0 - 0,0
1,Australia,0,0,0,0 - 0,0
2,Denmark,0,0,0,0 - 0,0
3,Tunisia,0,0,0,0 - 0,0


In [110]:
dict_table[6]

,Team,W,D,L,GF - GA,Pts
0,Spain,0,0,0,0 - 0,0
1,Costa Rica,0,0,0,0 - 0,0
2,Germany,0,0,0,0 - 0,0
3,Japan,0,0,0,0 - 0,0


In [111]:
dict_table[7]

,Team,W,D,L,GF - GA,Pts
0,Belgium,0,0,0,0 - 0,0
1,Canada,0,0,0,0 - 0,0
2,Morocco,0,0,0,0 - 0,0
3,Croatia,0,0,0,0 - 0,0


In [112]:
dict_table[8]

,Team,W,D,L,GF - GA,Pts
0,Brazil,0,0,0,0 - 0,0
1,Serbia,0,0,0,0 - 0,0
2,Switzerland,0,0,0,0 - 0,0
3,Cameroon,0,0,0,0 - 0,0


In [113]:
dict_table[9]

,Team,W,D,L,GF - GA,Pts
0,Portugal,0,0,0,0 - 0,0
1,Ghana,0,0,0,0 - 0,0
2,Uruguay,0,0,0,0 - 0,0
3,South Korea,0,0,0,0 - 0,0


In [114]:
for dict_table[2] in dict_table:
    print(dict_table[2].values)

[[nan nan 'Qatar' nan 'Ecuador' 'A']
 [nan nan 'Senegal' nan 'Netherlands' 'A']
 [nan nan 'England' nan 'Iran' 'B']
 [nan nan 'United States' nan 'Wales' 'B']
 [nan nan 'Argentina' nan 'Saudi Arabia' 'C']
 [nan nan 'Denmark' nan 'Tunisia' 'D']
 [nan nan 'Mexico' nan 'Poland' 'C']
 [nan nan 'France' nan 'Australia' 'D']
 [nan nan 'Morocco' nan 'Croatia' 'F']
 [nan nan 'Germany' nan 'Japan' 'E']
 [nan nan 'Spain' nan 'Costa Rica' 'E']
 [nan nan 'Belgium' nan 'Canada' 'F']
 [nan nan 'Switzerland' nan 'Cameroon' 'G']
 [nan nan 'Uruguay' nan 'South Korea' 'H']
 [nan nan 'Portugal' nan 'Ghana' 'H']
 [nan nan 'Brazil' nan 'Serbia' 'G']
 [nan nan 'Wales' nan 'Iran' 'B']
 [nan nan 'Qatar' nan 'Senegal' 'A']
 [nan nan 'Netherlands' nan 'Ecuador' 'A']
 [nan nan 'England' nan 'United States' 'B']
 [nan nan 'Tunisia' nan 'Australia' 'D']
 [nan nan 'Poland' nan 'Saudi Arabia' 'C']
 [nan nan 'France' nan 'Denmark' 'D']
 [nan nan 'Argentina' nan 'Mexico' 'C']
 [nan nan 'Japan' nan 'Costa Rica' 'E']
 [

In [ ]:
# run all the matches in the group stage and update group tables